In [1]:
import pandas as pd
import numpy as np
import sqlite3
from pandasql import sqldf
pysqldf=lambda q :sqldf(q,globals())

In [2]:
# pip install pysqldf

<b> Testing the waters </b>

In [3]:
pd.options.display.max_colwidth=100
Keywords=pd.read_csv("Keywords.csv")
Keywords['Topic']=Keywords['Topic'].apply(lambda x:x.upper())
Keywords.head(),Keywords.tail()

(         ID                 Topic  Parent Topic
 0  46485368         SYSTEM DESIGN      46485367
 1  46485369       SYSTEM ANALYSIS      46485367
 2  46485370  WORK DOMAIN ANALYSIS      46485367
 3  46485355     DISTANCE LEARNING      46485354
 4  46485356  DISTRIBUTED TRAINING      46485354,
            ID                                       Topic  Parent Topic
 355  46485310                         SPECIAL POPULATIONS             0
 356  46485315                      SURFACE TRANSPORTATION             0
 357  46485367        SYSTEM DESIGN AND ANALYSIS (GENERAL)             0
 358  46485343                            TEAMS AND GROUPS             0
 359  46485354  TRAINING, EDUCATION, INSTRUCTIONAL SYSTEMS             0)

In [4]:
conditions=[Keywords["Parent Topic"]==0,
           Keywords["Parent Topic"]>0]
choices=["Primary","Secondary"]
Keywords["Skill_Type"]=np.select(conditions,choices)
Keywords.loc[Keywords.ID==46485115,"Topic"]="VIGILANCE (SUSTAINED ATTENTION), MONITORING, SUPERVISORY CONTROL"

Keywords

,ID,Topic,Parent Topic,Skill_Type
0,46485368,SYSTEM DESIGN,46485367,Secondary
1,46485369,SYSTEM ANALYSIS,46485367,Secondary
2,46485370,WORK DOMAIN ANALYSIS,46485367,Secondary
3,46485355,DISTANCE LEARNING,46485354,Secondary
4,46485356,DISTRIBUTED TRAINING,46485354,Secondary
...,...,...,...,...
355,46485310,SPECIAL POPULATIONS,0,Primary
356,46485315,SURFACE TRANSPORTATION,0,Primary
357,46485367,SYSTEM DESIGN AND ANALYSIS (GENERAL),0,Primary
358,46485343,TEAMS AND GROUPS,0,Primary


In [5]:
Info=pd.read_csv("Active_User_Information_for_Export.csv")
Info=Info.fillna(0)
#dropping rows with Primary Attribute=0
Info=Info[Info["Primary Attribute"]!=0]
#dropping Rows containing short irrelevant entries
Info=Info[Info["Primary Attribute"].str.len()>3]
#dropping Rows with Opt In
Info=Info[Info["Primary Attribute"]!='Opt In']
Info=Info[Info["Primary Attribute"]!='Opt Out']

#capitalizing Primary Attribute
Info["Primary Attribute"]=Info["Primary Attribute"].apply(lambda x:x.upper())
Info

C:\Users\Patron\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4,7,8,9,10,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Salutation,First Name,Middle Name,Last Name,Suffix,Degree,Primary E-Mail Address,Secondary E-Mail Address,Phone,Fax,...,State,Country/Region,Postal Code,Secondary Attribute,Primary Attribute,Manuscript ID,Manuscript Type,Date Assigned,Task Name,Role
0,Prof. ...,Jonathan,0,Schooler,0,0,schooler@psych.ucsb.edu,0,0,0,...,California,United States,93105,Area of Expertise,METACOGNITION,HF-12-3710.R1,Research Article,4/14/2013 18:20,Reviewers Score,Reviewer
1,Prof. ...,Jonathan,0,Schooler,0,0,schooler@psych.ucsb.edu,0,0,0,...,California,United States,93105,Area of Expertise,ATTENTION,HF-12-3710.R1,Research Article,4/14/2013 18:20,Reviewers Score,Reviewer
2,Prof. ...,Jonathan,0,Schooler,0,0,schooler@psych.ucsb.edu,0,0,0,...,California,United States,93105,Area of Expertise,MEMORY,HF-12-3710.R1,Research Article,4/14/2013 18:20,Reviewers Score,Reviewer
5,Dr,Mohammad,0,Abdoli,0,0,m.abdoli@ryerson.ca,0,4169795000 X7624,0,...,Ontario,Canada,M5B2K3,Keywords,ANTHROPOMETRY,HF-20-6706,Research Article,4/30/2020 9:17,Reviewers Score,Reviewer
6,Dr,Mohammad,0,Abdoli,0,0,m.abdoli@ryerson.ca,0,4169795000 X7624,0,...,Ontario,Canada,M5B2K3,Area of Expertise,"BIOMECHANICS, WORK PHYSIOLOGY, ANTHROPOMETRY",HF-20-6706,Research Article,4/30/2020 9:17,Reviewers Score,Reviewer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164012,Dr,Elif,0,�zcan,0,0,E.Ozcan@tudelft.nl,0,31152789678,0,...,Zuid-Holland,Netherlands,2628 CE,Keywords,MEDICAL DEVICES AND TECHNOLOGIES,HF-20-6583.R1,Research Article,7/23/2020 13:53,Reviewers Score,Reviewer
164013,Dr,Elif,0,�zcan,0,0,E.Ozcan@tudelft.nl,0,31152789678,0,...,Zuid-Holland,Netherlands,2628 CE,Keywords,NURSING AND NURSING SYSTEMS,HF-20-6583.R1,Research Article,7/23/2020 13:53,Reviewers Score,Reviewer
164014,Dr,Elif,0,�zcan,0,0,E.Ozcan@tudelft.nl,0,31152789678,0,...,Zuid-Holland,Netherlands,2628 CE,Keywords,PATIENT SAFETY,HF-20-6583.R1,Research Article,7/23/2020 13:53,Reviewers Score,Reviewer
164015,Dr,Elif,0,�zcan,0,0,E.Ozcan@tudelft.nl,0,31152789678,0,...,Zuid-Holland,Netherlands,2628 CE,Keywords,PRODUCT DESIGN,HF-20-6583.R1,Research Article,7/23/2020 13:53,Reviewers Score,Reviewer


In [6]:
User_Data=Info.drop(columns=["Manuscript ID","Manuscript Type","Date Assigned","Task Name","Secondary Attribute","Primary Attribute","Phone"])
User_Data.drop_duplicates("Primary E-Mail Address",keep='first',inplace=True)
User_Data.dtypes

Salutation                  object
First Name                  object
Middle Name                 object
Last Name                   object
Suffix                      object
Degree                      object
Primary E-Mail Address      object
Secondary E-Mail Address    object
Fax                         object
Unavailable Dates           object
Institution                 object
Department                  object
Address 1                   object
Address 2                   object
City                        object
State                       object
Country/Region              object
Postal Code                 object
Role                        object
dtype: object

In [7]:
User_Skill=Info[["Primary E-Mail Address","Primary Attribute"]]
#Split the rows with Primary attribute containing <
#https://stackoverflow.com/questions/50731229/split-cell-into-multiple-rows-in-pandas-dataframe
User_Skill.set_index(['Primary E-Mail Address'],inplace=True)
User_Skill=User_Skill.apply(lambda x:x.str.split('<').explode())

#trimming space before string
User_Skill["Primary Attribute"]=User_Skill["Primary Attribute"].str.strip()
User_Skill=User_Skill.reset_index()

#replacing certain values manually
User_Skill["Primary Attribute"]=User_Skill["Primary Attribute"].replace(
            [
                'NOISE/ACOUSTIC',
                'ARCHITECTURAL',
                'CONSUMER PRODUCTS',
                'KINESTHETIC, ORIENTATION, BALANCE PERCEPTION',
                'PHYSIOLOGICAL AND PSYCHOLOGICAL CONDITIONS AND -INTERNAL ENVIRONMENT-',
                'ANALYSIS AND EVALUATION TECHNIQUES (E.G., SAINT)',
                'AUDITION (BASIC)',
                'COMMUNICATION SYSTEMS',
                'CUMULATIVE TRAUMA DISORDERS',
                'DECISION MAKING, JUDGMENT, RISK ASSESSMENT',
                'DISPLAY-CONTROL COMPATIBILITY',
                'DRIVER BEHAVIOUR',
                'VIGILANCE (SUSTAINED ATTENTION), MONITORING, SUPERVISORY CONTROL',
                'EYE MOVEMENT, TRACKING',
                'AVIATION',
                'CONTROLLED PROCESSING/AUTOMATIC PROCESSING',
                'DRIVER BEHAVIOUR',
                'INDUSTRIAL/ORGANIZATIONAL/WORKPLACE',
                'MUSCULOSKELETAL DISORDERS'
            ],
            [
                'NOISE/ACOUSTICS',
                'ARCHITECTURE',
                'CONSUMER PRODUCTS, TOOLS',
                'KINESTHESIS,PROPRIOCEPTION, ORIENTATION, BALANCE',
                'PHYSIOLOGICAL AND PSYCHOLOGICAL CONDITIONS ( -INTERNAL ENVIRONMENT-)',
                'ANALYSIS AND EVALUATION',
                'AUDITION',
                'COMMUNICATION',
                'MUSCULOSKELETAL SYSTEM (MUSCULOSKELETAL DISORDERS, CUMULATIVE TRAUMA DISORDER)',
                'DECISION MAKING',
                'DISPLAY-CONTROL OR STIMULUS-RESPONSE COMPATIBILITY',
                'DRIVER BEHAVIOR',
                'VIGILANCE (SUSTAINED ATTENTION), MONITORING, SUPERVISORY CONTROL""',
                'EYE MOVEMENTS, TRACKING',
                'AVIATION AND AEROSPACE',
                'AUTOMATIC AND CONTROLLED PROCESSING',
                'DRIVER BEHAVIOR',
                'INDUSTRIAL/WORKPLACE ERGONOMICS',
                'MUSCULOSKELETAL SYSTEM (MUSCULOSKELETAL DISORDERS, CUMULATIVE TRAUMA DISORDER)'
            ])

Recorded_Skills=["AGE AND AGING EFFECTS","AIRCREW BEHAVIOR","AMBIENT CONDITIONS AND PROTECTION FROM/ADAPTATION TO . . .","AUTOMATICITY","COGNITION: SITUATION AWARENESS","COGNITIVE ENGINEERING","COGNITIVE TASK ANALYSIS","COMPUTER SYSTEMS","COMPUTER SYSTEMS: INTERFACE EVALUATION, USABILITY","CONTROLS","CONTROLS: GRAPHICAL USER INTERFACES (GUI)","DISABILITY/HANDICAP/FUNCTIONAL IMPAIRMENT","DISPLAYS","DISTANCE/DISTRIBUTED LEARNING AND TRAINING","DRIVING","ECOLOGICAL VALIDITY","EDUCATION","ENVIRONMENT","ENVIRONMENT/ENVIRONMENT/CONTEXT","FALLS","HEALTH CARE","HIGHWAY AND VEHICLE DESIGN","HIGHWAY/AUTOMOBILE","HUMAN ERROR: AVIATION","HUMAN ERROR: DRIVING","INTERFACE EVALUATION, USABILITY","LEARNING/TRANSFER/SKILL MAINTENANCE (BASIC)","LIGHT AND LIGHTING/OPTIC","LONG-TERM MEMORY","LOW BACK INJURIES","MANUFACTURING","MARITIME ISSUES","MEDICAL/HEALTH","MENTAL","MENTAL MEASUREMENT (E.G., PSYCHOPHYSICS, SCALING)","MENTAL MODELS, SHARED MENTAL MODELS,","METHODOLOGICAL (GENERAL)","MODELS AND MEASURES","MULTIPLE RESOURCES","PHYSIOLOGICAL MEASURES, MODELS","PROBLEMSOLVING, REASONING","PSYCHOMETRICS AND TESTING","PSYCHOMOTOR PROCESSES","PSYCHOPHYSIOLOGY","RESEARCH DESIGN (EXPERIMENTAL, QUASIEXPERIMENTAL, FIELD)","RISK MANAGEMENT","SENSORY/PERCEPTUAL PROCESSES","SIMULATION-BASED ACQUISITION","SKILL ACQUISITION","SKILLED PERFORMANCE/PERCEPTUAL-MOTOR SKILL","SOCIOTECHNICAL SYSTEMS, MACROERGONOMICS, AND THE ENVIRONMENT","SPACE","SUPERVISORY (E.G., PROCESS CONTROL, AUTOMATED SYSTEMS)","TACTILE DISPLAYS","TACTUAL/HAPTIC DISPLAYS","TACTUAL/HAPTIC PERCEPTION","TEAM PERFORMANCE","TRACKBALLS, MICE, JOYSTICKS, OTHER OUTPUT DEVICES","TRAINING","TRAINING TECHNOLOGIES (E.G., WEB-BASED)","TRUST","VIGILANCE (SUSTAINED ATTENTION), MONITORING, SUPERVISORY CONTROL"
                ]
To_change_to=["Aging Processes","Pilot, crew behavior","PHYSICAL/AMBIENT ENVIRONMENT","Levels of automation","Situation awareness","COGNITION","Task Analysis","HUMAN-COMPUTER INTERACTION, COMPUTER SYSTEMS","INTERFACE EVALUATION","DISPLAYS AND CONTROLS","Graphical user interfaces (GUI)","SPECIAL POPULATIONS","DISPLAYS AND CONTROLS","Distance Learning","Driver behavior","ECOLOGICAL APPROACHES (E.G., ECOLOGICAL INTERFACE DESIGN)","TRAINING, EDUCATION, INSTRUCTIONAL SYSTEMS","ENVIRONMENTAL DESIGN","ENVIRONMENTAL DESIGN","Slips and falls","HEALTH CARE/HEALTH SYSTEMS","Vehicle design","Highway systems","PILOT DECISION MAKING","Driver behavior","Interface evaluation","TRAINING, EDUCATION, INSTRUCTIONAL SYSTEMS","WORKSPACE, WORKSTATION, -BUILT ENVIRONMENT- DESIGN","MEMORY (SHORT-TERM, LONG-TERM, WORKING MEMORY)","MUSCULOSKELETAL SYSTEM (MUSCULOSKELETAL DISORDERS, CUMULATIVE TRAUMA DISORDER)","MANUFACTURING, PROCESS CONTROL SYSTEMS","Maritime systems","HEALTH CARE/HEATH SYSTEMS","COGNITION","SENSORY AND PERCEPTUAL PROCESSES","Mental models, shared mental models","METHODS AND SKILLS","MATHEMATICAL MODELING","Multiple resource models","PHYSIOLOGICAL AND PSYCHOLOGICAL CONDITIONS ( -INTERNAL ENVIRONMENT-)","PROBLEM-SOLVING","PSYCHOMETRICS, SCALING","PERCEPTUAL-MOTOR PERFORMANCE","PSYCHOPHYSICAL METHODS","Experimental design","Risk assessment","SENSORY AND PERCEPTUAL PROCESSES","SIMULATION AND VIRTUAL REALITY","Motor learning","Skilled performance","MACROERGONOMICS AND THE ENVIRONMENT","AVIATION AND AEROSPACE","Supervisory control","Tactile/haptic displays","Tactile/haptic displays","Haptic/touch","TEAMS AND GROUPS","Trackballs, mice, joysticks, etc.","TRAINING, EDUCATION, INSTRUCTIONAL SYSTEMS","Instructional technologies","Trust in automation","Supervisory control"
             ]

User_Skill["Primary Attribute"]=User_Skill["Primary Attribute"].replace(
            Recorded_Skills,
            To_change_to)

#replacing jumbled up skills with exact match
for attribute in User_Skill["Primary Attribute"]:
    attribute=attribute.upper()
    if attribute not in Keywords["Topic"].to_list():
        b=sorted(attribute.split())
        for keyword in Keywords["Topic"].to_list():
            a=sorted(keyword.split())
            if a==b:
                User_Skill["Primary Attribute"].replace(
                    to_replace=attribute,
                    value=keyword,
                    inplace=True)
                
#replacing jumbled up skills with(and comma) exact match

for attribute in User_Skill["Primary Attribute"]:
    attribute=attribute.upper()
    if attribute not in Keywords["Topic"].to_list():
        b=sorted(attribute.split(', '))
        for keyword in Keywords["Topic"].to_list():
            a=sorted(keyword.split(', '))
            if a==b:
                User_Skill["Primary Attribute"].replace(
                    to_replace=attribute,
                    value=keyword,
                    inplace=True)
    
#removing duplicates
User_Skill=User_Skill.drop_duplicates()

#matching using regex
User_Skill.replace(
            to_replace=[r'^HCI.*',r'CONTEXT.*',r'ATTENTION.*',r'ACCIDENTS.*',r'DUAL-TASK.*',r'VIGILANCE.*',
                       r'^AEROSPACE.*',r'^AUDITION.*',r'^DECISION MAKING.*',r'^VISUAL .*',
                       r'^TRANSPORTATION.*',r'^NOVICE.*',r'^MUSCULOSKELETAL.*',r'^MEMORY.*'],
            value=['HUMAN-COMPUTER INTERACTION, COMPUTER SYSTEMS',
                   'ENVIRONMENT/CONTEXT',
                   'ATTENTIONAL PROCESSES',
                   'ACCIDENTS,  HUMAN ERROR',
                   'DUAL TASK, TIME SHARING, TASK SWITCHING',
                   'VIGILANCE (SUSTAINED ATTENTION), MONITORING, SUPERVISORY CONTROL',
                  'AVIATION AND AEROSPACE',
                    'AUDITION',
                    'DECISION MAKING',
                  'VISUAL SEARCH',
                  'SURFACE TRANSPORTATION',
                  'EXPERT-NOVICE DIFFERENCES',
                  'MUSCULOSKELETAL SYSTEM (MUSCULOSKELETAL DISORDERS, CUMULATIVE TRAUMA DISORDER)',
                   'MEMORY (SHORT-TERM, LONG-TERM, WORKING MEMORY)'
                  ],
                    regex=True,
                    inplace=True)

User_Skill["Primary Attribute"]=User_Skill["Primary Attribute"].replace(["ENVIRONMENT/ENVIRONMENT/CONTEXT","HEALTH CARE/HEATH SYSTEMS"],
                                                      ["ENVIRONMENTAL DESIGN","HEALTH CARE/HEALTH SYSTEMS"])

User_Skill

,Primary E-Mail Address,Primary Attribute
0,schooler@psych.ucsb.edu,METACOGNITION
1,schooler@psych.ucsb.edu,ATTENTIONAL PROCESSES
2,schooler@psych.ucsb.edu,"MEMORY (SHORT-TERM, LONG-TERM, WORKING MEMORY)"
3,m.abdoli@ryerson.ca,ANTHROPOMETRY
4,m.abdoli@ryerson.ca,"BIOMECHANICS, ANTHROPOMETRY, WORK PHYSIOLOGY"
...,...,...
167098,E.Ozcan@tudelft.nl,MEDICAL DEVICES AND TECHNOLOGIES
167099,E.Ozcan@tudelft.nl,NURSING AND NURSING SYSTEMS
167100,E.Ozcan@tudelft.nl,PATIENT SAFETY
167101,E.Ozcan@tudelft.nl,PRODUCT DESIGN


<b> Creating a database </b>

In [8]:
conn=sqlite3.connect('DataBase.sqlite',timeout=20)
cur=conn.cursor()
cur.executescript('''

DROP TABLE IF EXISTS User_Data;
DROP TABLE IF EXISTS Skill;
DROP TABLE IF EXISTS Keywords;

CREATE TABLE User_Data(
                    User_ID INTEGER PRIMARY KEY AUTOINCREMENT,
                    Salutation VARCHAR(255),
                    First_Name VARCHAR(255),
                    Middle_Name VARCHAR(255),
                    Last_Name VARCHAR(255),
                    Suffix VARCHAR(255),
                    Degree VARCHAR(255),
                    Primary_EMail_Address VARCHAR(255) UNIQUE,
                    Secondary_EMail_Address VARCHAR(255),
                    Fax VARCHAR(255),
                    Unavailable_Dates VARCHAR(255),
                    Institution VARCHAR(255),
                    Department VARCHAR(255),
                    Address1 VARCHAR(255),
                    Address2 VARCHAR(255),
                    City VARCHAR(255),
                    State VARCHAR(255),
                    CountryRegion VARCHAR(255),
                    Postal_Code VARCHAR(255),
                    Role VARCHAR(255)
);

CREATE TABLE Keywords(
                    Skill_ID INTEGER PRIMARY KEY NOT NULL UNIQUE,
                    Topic VARCHAR(255),
                    Parent_Topic VARCHAR(255),
                    Skill_type VARCHAR(255)
                    );
                    
CREATE TABLE Skill(
                    ID INTEGER PRIMARY KEY AUTOINCREMENT,
                    User_ID INTEGER,
                    Skill_ID INTEGER,
                    FOREIGN KEY(User_ID) REFERENCES User_Data(User_ID),
                    FOREIGN KEY(Skill_ID) REFERENCES Keywords(Skill_ID)
                    )
                    

''')
conn.commit()

In [9]:
for index,row in User_Data.iterrows():
    list_row=[]
    for column in User_Data.columns.to_list():
        if row[column]==0:
            list_row.append(None) 
        else:
            list_row.append(row[column])
    tuple_row=tuple(list_row)
    cur.execute('''INSERT OR IGNORE INTO User_Data(Salutation,First_Name ,Middle_Name ,Last_Name,Suffix,Degree,Primary_EMail_Address,Secondary_EMail_Address,Fax,Unavailable_Dates,Institution,Department,Address1,Address2,City,State,CountryRegion,Postal_Code,Role)
    VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''',tuple_row)

conn.commit()


In [10]:
for index,row in Keywords.iterrows():
    list_row=[]
    for column in Keywords.columns.to_list():
        list_row.append(row[column])
    tuple_row=tuple(list_row)
    cur.execute('''INSERT OR IGNORE INTO Keywords(Skill_ID,Topic,Parent_Topic,Skill_Type) VALUES(?,?,?,?)''',tuple_row)

cur.execute('''INSERT OR IGNORE INTO Keywords(Skill_ID,Topic,Parent_Topic,Skill_Type) VALUES(1,"Not recognizable",1,"NA")''')
cur.execute('''INSERT OR IGNORE INTO Keywords(Skill_ID,Topic,Parent_Topic,Skill_Type) VALUES(0,"Keywords","Keywords","Keywords")''')

conn.commit()
tuple_row

(46485354, 'TRAINING, EDUCATION, INSTRUCTIONAL SYSTEMS', 0, 'Primary')

In [11]:
unrecognizable=[]
for index,row in User_Skill.iterrows():
    cur.execute('''SELECT User_ID FROM User_Data WHERE Primary_EMail_Address=?''',(row["Primary E-Mail Address"],))
    User_ID=cur.fetchone()[0]
    try:
        cur.execute('''SELECT Skill_ID FROM Keywords WHERE Topic=?''',(row['Primary Attribute'].upper(),))
        Skill_ID=cur.fetchone()[0] 
    except:
        Skill_ID=1
        if row["Primary Attribute"] not in unrecognizable:
            unrecognizable.append(row["Primary Attribute"])

    cur.execute('''INSERT OR IGNORE INTO Skill(User_ID,Skill_ID) VALUES(?,?)''',(User_ID,Skill_ID))

conn.commit()
sorted(unrecognizable), len(unrecognizable)

(['EXPERTISE',
  'FORENSICS',
  'FUNCTION ALLOCATION IN SYSTEM DESIGN',
  'GENERAL PRINCIPLES',
  'GENERAL PRINCIPLES (E.G., COMPATIBILITY, CODING, STEREOTYPES)',
  'GROUPWARE',
  'HOME',
  'HUMAN ERROR: MEDICINE',
  'INTERNET',
  'INTERRUPTION',
  'LEADERSHIP',
  'MANUAL',
  'MICROERGONOMICS',
  'MILITARY',
  'PERFORMANCE-ORIENTED TOPICS',
  'PHYSICAL',
  'SYSTEM DESIGN FEATURES',
  'TELECONFERENCING',
  'TIME TO CONTACT',
  'USER CENTERED DESIGN',
  'WORKLOAD'],
 21)

<b> Generate Excel Sheets with connections</b>

<b> Connect to the master list
</b>

In [12]:
Current_AE=pd.read_excel("Editorial Board.xlsx",sheet_name='Associate Editors')
Current_AE=Current_AE[["Name","Email ID","Affiliation"]]
Current_AE

,Name,Email ID,Affiliation
0,"Cao, Caroline",caroline.cao@imt-atlantique.fr,Wright State University
1,"Cooke, Nancy",nancy.cooke@asu.edu,Arizona State University
2,"Davis, Kermit G.",kermit.davis@uc.edu,University of Cincinnati
3,"Draper, Mark H.",mark.draper.2@us.af.mil,U.S. Air Force Research Laboratory
4,"Feng, Jing",jing_feng@ncsu.edu,North Carolina State University
5,"Gallagher, Sean",seangallagher@auburn.edu,Auburn University
6,"Goettl, Barry P.",barry.goettl@us.af.mil,U.S. Air Force Research Laboratory
7,"Greenlee, Eric",eric.greenlee@ttu.edu,Texas Tech University
8,"Hollands, Justin G.",justin.hollands@drdc-rddc.gc.ca,DRDC Toronto
9,"Horrey, William J.",william.horrey@gmail.com,AAA Foundation for Traffic Safety


In [13]:
Current_EB=pd.read_excel("Editorial Board.xlsx",sheet_name='Editorial Board')
Current_EB=Current_EB[["Name","Email ID","Affiliation"]]
Current_EB

,Name,Email ID,Affiliation
0,"Armstrong, Thomas J.",tja@umich.edu,University of Michigan
1,"Bell, Herbert H.",hbbb46@gmail.com,Discerning Technologies
2,"Beringer, Dennis B.",dr_who@swbell.net,FAA Civil Aerospace Medical Institute
3,"Bisantz, Ann M.",bisantz@buffalo.edu,"University of Buffalo, The State University of New York"
4,"Boyle, Linda Ng",linda@uw.edu,University of Washington
5,"Calhoun, Gloria L.",dragonfly5119@yahoo.com,U.S. Air Force
6,"Czaja, Sara J.",SCzaja@med.miami.edu,Cornell University
7,"de Winter, Joost",j.c.f.dewinter@tudelft.nl,Delft University of Technology
8,"Drews, Frank A.",frank.drews@psych.utah.edu,University of Utah
9,"Gorman, Jamie C.",jamie.gorman@gatech.edu,Georgia Institute of Technology


In [14]:
Current_PR=pd.read_excel("Editorial Board.xlsx",sheet_name='Preferred Reviewers')
Current_PR=Current_PR[["Name","Email ID","Affiliation"]]
Current_PR.dropna(inplace=True)
Current_PR

,Name,Email ID,Affiliation
1,"Chen, Huei-Yen Winnie",winchen@buffalo.edu,University at Buffalo
2,"Chen, Karen Boru",kbchen2@ncsu.edu,North Carolina State University
3,"Chowdhury, Suman",suman.chowdhury@ttu.edu,Texas Tech University
4,"Gutzwiller, Robert S.",robert.gutzwiller@gmail.com,Arizona State University
6,"McDonald, Tony",mcdonald@tamu.edu,Texas A&M University
7,"Park, Woojin",woojinpark@snu.ac.kr,Seoul National University
8,"Stirling, Leia",leias@umich.edu,University of Michigan
9,"Wilson, Michael D.",michael.d.wilson@curtin.edu.au,Curtin University


In [15]:
#adding Current_AE in DB
cur.executescript('''
DROP TABLE IF EXISTS AE;

CREATE TABLE AE(
                ID INTEGER PRIMARY KEY AUTOINCREMENT,
                User_ID INTEGER
                    )
''')

for index,row in Current_AE.iterrows():
    print(row["Email ID"])
    cur.execute('''SELECT User_ID FROM User_Data WHERE Primary_EMail_Address=?''',(row["Email ID"],))
#    print(cur.fetchone())
    User_ID=cur.fetchone()[0]
    print(User_ID)
    cur.execute('''INSERT OR IGNORE INTO AE(User_ID) VALUES(?)''',(User_ID,))

conn.commit()

caroline.cao@imt-atlantique.fr
115
nancy.cooke@asu.edu
162
kermit.davis@uc.edu
180
mark.draper.2@us.af.mil
199
jing_feng@ncsu.edu
225
seangallagher@auburn.edu
246
barry.goettl@us.af.mil
264
eric.greenlee@ttu.edu
274
justin.hollands@drdc-rddc.gc.ca
315
william.horrey@gmail.com
318
jdlee@engr.wisc.edu
416
frank.ritter@psu.edu
578
sarter@umich.edu
599
sommerich.1@osu.edu
633
yyamani@odu.edu
738
xudongzhang@tamu.edu
750


In [16]:
#adding Current_EB in DB
cur.executescript('''
DROP TABLE IF EXISTS EB;

CREATE TABLE EB(
                ID INTEGER PRIMARY KEY AUTOINCREMENT,
                User_ID INTEGER
                    )
''')

for index,row in Current_EB.iterrows():
    print(row["Email ID"])
    cur.execute('''SELECT User_ID FROM User_Data WHERE Primary_EMail_Address=?''',(row["Email ID"],))
#     print(cur.fetchone())
    User_ID=cur.fetchone()[0]
    print(User_ID)
    cur.execute('''INSERT OR IGNORE INTO EB(User_ID) VALUES(?)''',(User_ID,))

conn.commit()

tja@umich.edu
28
hbbb46@gmail.com
53
dr_who@swbell.net
60
bisantz@buffalo.edu
72
linda@uw.edu
89
dragonfly5119@yahoo.com
112
SCzaja@med.miami.edu
170
j.c.f.dewinter@tudelft.nl
724
frank.drews@psych.utah.edu
201
jamie.gorman@gatech.edu
270
whelton@gmu.edu
305
michael.jorgensen@wichita.edu
347
kirlik@illinois.edu
382
lilaflaux@gmail.com
409
lavender.1@osu.edu
410
marquard@umn.edu
456
martinbj@umich.edu
462
chris_mayhorn@ncsu.edu
466
mcneese@clemson.edu
475
sherry.mead@hpe.com
476
jmeyer@tau.ac.il
483
mirka@iastate.edu
493
aparush@ie.technion.ac.il
535
rproctor@purdue.edu
552
esa.rantanen@rit.edu
562
emroth@mindspring.com
585
Eduardo.salas@rice.edu
590
p.sanderson@uq.edu.au
596
mark-schall@auburn.edu
603
frank.schieber@usd.edu
605
stjohn@pacific-science.com
639
kvu8@csulb.edu
689
pandawickens94@aol.com
716
dawiegmann@wisc.edu
717
gfwilson@outlook.com
720
yan.xiao.phd@gmail.com
732


In [17]:
#adding Current_PR in DB
cur.executescript('''
DROP TABLE IF EXISTS PR;

CREATE TABLE PR(
                ID INTEGER PRIMARY KEY AUTOINCREMENT,
                User_ID INTEGER
                    )
''')

for index,row in Current_PR.iterrows():
    cur.execute('''SELECT User_ID FROM User_Data WHERE Primary_EMail_Address=?''',(row["Email ID"],))
#     print(cur.fetchone())
    print(row["Email ID"])
    User_ID=cur.fetchone()[0]
    print(User_ID)
    cur.execute('''INSERT OR IGNORE INTO PR(User_ID) VALUES(?)''',(User_ID,))

conn.commit()

winchen@buffalo.edu
135
kbchen2@ncsu.edu
139
suman.chowdhury@ttu.edu
148
robert.gutzwiller@gmail.com
282
mcdonald@tamu.edu
469
woojinpark@snu.ac.kr
534
leias@umich.edu
645
michael.d.wilson@curtin.edu.au
721


In [18]:
# AE combined
conn=sqlite3.connect('DataBase.sqlite')
Keywords=pd.read_sql_query("SELECT *  FROM Keywords",conn)
Skill=pd.read_sql_query("SELECT *  FROM Skill",conn)
User_Data=pd.read_sql_query("SELECT *  FROM User_Data",conn)
AE=pd.read_sql_query("SELECT *  FROM AE",conn)

Join_Skill_AE=pysqldf(
            '''
            select User_Data.User_ID,Salutation,(select First_Name||" "|| Last_Name) as Name,
            Primary_EMail_Address, 
            Keywords.Topic as Area_of_Expertise,
            Skill_type as Level_of_Skill,
            Table1.Topic as Keywords
            from AE
            join Skill on AE.User_ID=Skill.User_ID
            join User_Data on Skill.User_ID=User_Data.User_ID
            join Keywords on Skill.Skill_ID=Keywords.Skill_ID
            left join (SELECT Keywords.Skill_ID,Keywords.Topic from Keywords) as Table1 
            on Table1.Skill_ID=Keywords.Parent_Topic
'''
)

            
Join_Skill_AE.to_excel("Sample_output_AE.xlsx")
Join_Skill_AE

,User_ID,Salutation,Name,Primary_EMail_Address,Area_of_Expertise,Level_of_Skill,Keywords
0,115,Dr,Caroline Cao,caroline.cao@imt-atlantique.fr,Not recognizable,NA,Not recognizable
1,115,Dr,Caroline Cao,caroline.cao@imt-atlantique.fr,MULTIPLE RESOURCE MODELS,Secondary,COGNITION
2,115,Dr,Caroline Cao,caroline.cao@imt-atlantique.fr,INTERFACE EVALUATION,Secondary,"HUMAN-COMPUTER INTERACTION, COMPUTER SYSTEMS"
3,115,Dr,Caroline Cao,caroline.cao@imt-atlantique.fr,HEALTH CARE/HEALTH SYSTEMS,Primary,Keywords
4,115,Dr,Caroline Cao,caroline.cao@imt-atlantique.fr,MEDICAL SIMULATION/TRAINING AND ASSESSMENT,Secondary,HEALTH CARE/HEALTH SYSTEMS
...,...,...,...,...,...,...,...
245,750,Professor,Xudong Zhang,xudongzhang@tamu.edu,"MUSCULOSKELETAL SYSTEM (MUSCULOSKELETAL DISORDERS, CUMULATIVE TRAUMA DISORDER)",Secondary,"BIOMECHANICS, ANTHROPOMETRY, WORK PHYSIOLOGY"
246,750,Professor,Xudong Zhang,xudongzhang@tamu.edu,HUMAN PERFORMANCE MODELING,Secondary,METHODS AND SKILLS
247,750,Professor,Xudong Zhang,xudongzhang@tamu.edu,MATHEMATICAL MODELING,Secondary,METHODS AND SKILLS
248,750,Professor,Xudong Zhang,xudongzhang@tamu.edu,STATISTICS AND DATA ANALYSIS,Secondary,METHODS AND SKILLS


In [23]:
# EB combined
conn=sqlite3.connect('DataBase.sqlite')
Keywords=pd.read_sql_query("SELECT *  FROM Keywords",conn)
Skill=pd.read_sql_query("SELECT *  FROM Skill",conn)
User_Data=pd.read_sql_query("SELECT *  FROM User_Data",conn)
EB=pd.read_sql_query("SELECT *  FROM EB",conn)

Join_Skill_EB=pysqldf(
            '''
            select User_Data.User_ID,Salutation,(select First_Name||" "|| Last_Name) as Name,
            Primary_EMail_Address, 
            Keywords.Topic as Area_of_Expertise,
            Skill_type as Level_of_Skill,
            Table1.Topic as Keywords
            from EB
            join Skill on EB.User_ID=Skill.User_ID
            join User_Data on Skill.User_ID=User_Data.User_ID
            join Keywords on Skill.Skill_ID=Keywords.Skill_ID
            left join (SELECT Keywords.Skill_ID,Keywords.Topic from Keywords) as Table1 
            on Table1.Skill_ID=Keywords.Parent_Topic
'''
)

            
Join_Skill_EB.to_excel("Output_EB.xlsx")
Join_Skill_EB

,User_ID,Salutation,Name,Primary_EMail_Address,Area_of_Expertise,Level_of_Skill,Keywords
0,28,Dr,Thomas Armstrong,tja@umich.edu,"BIOMECHANICS, ANTHROPOMETRY, WORK PHYSIOLOGY",Primary,Keywords
1,28,Dr,Thomas Armstrong,tja@umich.edu,KEYBOARDS,Secondary,DISPLAYS AND CONTROLS
2,28,Dr,Thomas Armstrong,tja@umich.edu,TACTILE/HAPTIC DISPLAYS,Secondary,DISPLAYS AND CONTROLS
3,28,Dr,Thomas Armstrong,tja@umich.edu,TOUCH SCREENS,Secondary,DISPLAYS AND CONTROLS
4,28,Dr,Thomas Armstrong,tja@umich.edu,"TRACKBALLS, MICE, JOYSTICKS, ETC.",Secondary,DISPLAYS AND CONTROLS
...,...,...,...,...,...,...,...
586,732,Dr,Yan Xiao,yan.xiao.phd@gmail.com,SIMULATION TRAINING,Secondary,HEALTH CARE/HEALTH SYSTEMS
587,732,Dr,Yan Xiao,yan.xiao.phd@gmail.com,"ORGANIZATIONAL BEHAVIOR/DESIGN, ORGANIZATIONAL PSYCHOLOGY",Secondary,MACROERGONOMICS AND THE ENVIRONMENT
588,732,Dr,Yan Xiao,yan.xiao.phd@gmail.com,TEAMS AND GROUPS,Primary,Keywords
589,732,Dr,Yan Xiao,yan.xiao.phd@gmail.com,COMMUNICATION ANALYSIS,Secondary,TEAMS AND GROUPS


In [24]:
# PR combined
conn=sqlite3.connect('DataBase.sqlite')
Keywords=pd.read_sql_query("SELECT *  FROM Keywords",conn)
Skill=pd.read_sql_query("SELECT *  FROM Skill",conn)
User_Data=pd.read_sql_query("SELECT *  FROM User_Data",conn)
PR=pd.read_sql_query("SELECT *  FROM PR",conn)

Join_Skill_PR=pysqldf(
            '''
            select User_Data.User_ID,Salutation,(select First_Name||" "|| Last_Name) as Name,
            Primary_EMail_Address, 
            Keywords.Topic as Area_of_Expertise,
            Skill_type as Level_of_Skill,
            Table1.Topic as Keywords
            from PR
            join Skill on PR.User_ID=Skill.User_ID
            join User_Data on Skill.User_ID=User_Data.User_ID
            join Keywords on Skill.Skill_ID=Keywords.Skill_ID
            left join (SELECT Keywords.Skill_ID,Keywords.Topic from Keywords) as Table1 
            on Table1.Skill_ID=Keywords.Parent_Topic
'''
)

            
Join_Skill_PR.to_excel("Sample_output_PR.xlsx")
Join_Skill_PR

,User_ID,Salutation,Name,Primary_EMail_Address,Area_of_Expertise,Level_of_Skill,Keywords
0,135,Professor,Huei-Yen Winnie Chen,winchen@buffalo.edu,HUMAN-AUTOMATION INTERACTION,Secondary,"AUTOMATION, EXPERT SYSTEMS"
1,135,Professor,Huei-Yen Winnie Chen,winchen@buffalo.edu,TECHNOLOGY ACCEPTANCE,Secondary,"AUTOMATION, EXPERT SYSTEMS"
2,135,Professor,Huei-Yen Winnie Chen,winchen@buffalo.edu,ATTENTIONAL PROCESSES,Secondary,COGNITION
3,135,Professor,Huei-Yen Winnie Chen,winchen@buffalo.edu,DISTRACTIONS AND INTERRUPTIONS,Secondary,COGNITION
4,135,Professor,Huei-Yen Winnie Chen,winchen@buffalo.edu,INFORMATION PROCESSING,Secondary,COGNITION
...,...,...,...,...,...,...,...
71,721,Dr,Michael David Wilson,michael.d.wilson@curtin.edu.au,SITUATION AWARENESS,Secondary,COGNITION
72,721,Dr,Michael David Wilson,michael.d.wilson@curtin.edu.au,JOB STRESS,Secondary,MACROERGONOMICS AND THE ENVIRONMENT
73,721,Dr,Michael David Wilson,michael.d.wilson@curtin.edu.au,COMPUTATIONAL MODELING,Secondary,METHODS AND SKILLS
74,721,Dr,Michael David Wilson,michael.d.wilson@curtin.edu.au,SIMULATION,Secondary,METHODS AND SKILLS
